# Function to generate a synthetic 3CH pma file from a 2CH pma file

In [1]:
import sys
import struct
sys.path.append('../')
from smpy import *

def synthetic_3CH_pma(orig_pma_file_path, new_pma_file_path):
    """
    Takes a 2-channel PMA file, creates and saves a synthetic 3-channel PMA file.
    """
    try:
        with open(orig_pma_file_path, "rb") as f:
            X_pixels, Y_pixels = struct.unpack("<HH", f.read(4))
            print(f"Image Size: {X_pixels} x {Y_pixels}")
            f.seek(0, 2) 
            filesize = f.tell()
            Nframes = (filesize - 4) // (X_pixels * Y_pixels)
            f.seek(4) 
            Frames_data_2CH = [np.frombuffer(f.read(X_pixels*Y_pixels), dtype=np.uint8).reshape((Y_pixels, X_pixels)) for frame_idx in range(Nframes)]
        
        Frames_data_3CH = []
        for frame_idx in range(Nframes):
            new_frame = np.zeros((Y_pixels, X_pixels), dtype=np.uint8)
            new_frame[:, :171] = Frames_data_2CH[frame_idx][:, :171]
            new_frame[:, 171:342] = Frames_data_2CH[frame_idx][:, 256:427]
            new_frame[:, 342:] = Frames_data_2CH[frame_idx][:, 256:426]
            Frames_data_3CH.append(new_frame)

        with open(new_pma_file_path, "wb") as f_out:
            f_out.write(struct.pack("<HH", X_pixels, Y_pixels))
            for frame in Frames_data_3CH:
                f_out.write(frame.astype(np.uint8).tobytes())
        print(f"Synthetic 3-channel PMA file sucessfully saved as {new_pma_file_path}")

    except Exception as e:
        print(f"Error reading .pma file: {e}")
        return None

